Courtesy of Andrej Karpathy

In [ ]:
from graphviz import Digraph

In [ ]:
from retrograd.engine import Value
from retrograd.neural import *

In [ ]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [ ]:
# a very simple example
# x = Value(4.0,label='x')
# y = (x*2 - 4*x + x**2)
# y.label='y'
# y.backward()
# draw_dot(y)

In [147]:
# MLP
x = [1. , 2. , 3.]
nn = MLP(3,[4,4,1])
y = nn(x)
# y.backward()
# draw_dot(y)

In [148]:
example_dataset = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0,1.0,-1.0],
]
y_real = [1.0,-1.0,-1.0,1.0]

for k in range(50):
  #forward pass
  y_pred = [nn(x) for x in example_dataset]
  loss = sum([(y_p - y_r)**2 for y_p,y_r in zip(y_pred,y_real)])

  #zero grad
  for p in nn.get_parameters():
    p.grad = 0.0

  #backward pass
  loss.backward()
  
  #update
  for p in nn.get_parameters():
    p.data -= 0.1 * p.grad
  print(f'Step {k} | Loss: {loss.data}')

Step 0 | Loss: 3.2238368630088936
Step 1 | Loss: 1.9598160010590902
Step 2 | Loss: 1.8114890777775585
Step 3 | Loss: 0.16664223646325327
Step 4 | Loss: 0.09369166696642
Step 5 | Loss: 0.06947609135004995
Step 6 | Loss: 0.055749660426420586
Step 7 | Loss: 0.04658099639073237
Step 8 | Loss: 0.039944641438023606
Step 9 | Loss: 0.03490109079790089
Step 10 | Loss: 0.030936339829231926
Step 11 | Loss: 0.027739545837245062
Step 12 | Loss: 0.02510973816866132
Step 13 | Loss: 0.02291057180417497
Step 14 | Loss: 0.02104607499446428
Step 15 | Loss: 0.019446675975033675
Step 16 | Loss: 0.01806070044862245
Step 17 | Loss: 0.01684897113698936
Step 18 | Loss: 0.01578125596525889
Step 19 | Loss: 0.014833863140520126
Step 20 | Loss: 0.013987971935537115
Step 21 | Loss: 0.01322844897196518
Step 22 | Loss: 0.012542992820269251
Step 23 | Loss: 0.011921505425317676
Step 24 | Loss: 0.01135562323825864
Step 25 | Loss: 0.010838362717706905
Step 26 | Loss: 0.010363848993306819
Step 27 | Loss: 0.009927105843145

In [149]:
y_pred

[Value( | Data=0.9723305923896031, Grad=-0.05533881522079387),
 Value( | Data=-0.9949161791963743, Grad=0.010167641607251365),
 Value( | Data=-0.9576602265879111, Grad=0.08467954682417789),
 Value( | Data=0.9503468269615258, Grad=-0.09930634607694833)]